In [1]:
import pandas as pd
import wandb

from features.extractor import FeatureExtractor
from configs import utils
utils.login_wandb()
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

/opt/homebrew/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/homebrew/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/homebrew/lib/python3.10/site-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keywo

In [2]:
sales = pd.read_excel('ucy_eko_data.xlsx', sheet_name='smile_sales')

In [3]:
fe = FeatureExtractor(sales, target_month=3)
X_train, X_test, y_train, y_test = fe.transform()

In [4]:
knn = KNeighborsClassifier()
config = knn.get_params()

# Create w&b run for the training set
with utils.init_wandb_run(
    name='initital_run',
    model=KNeighborsClassifier,
    config=config,
    group='default_parameters',
    job_type='train'
) as run:

    knn.fit(X_train, y_train)
    train_preds = knn.predict(X_train)

    acc = accuracy_score(y_train, train_preds)

    metadata = {
        'experiment': {
            'name': run.name,
        },
        'accuracy': acc,
        'config': config
    }

    artifact = wandb.Artifact(
        name=f'accuracy_train',
        type='performance_metric',
        metadata=metadata
    )
    run.log_artifact(artifact)
    run.finish()

# Create w&b run for the test set
with utils.init_wandb_run(
    name='initital_run',
    model=KNeighborsClassifier,
    config=config,
    group='default_parameters',
    job_type='test'
) as run:
    test_preds = knn.predict(X_test)
    test_acc = accuracy_score(y_test, test_preds)

    metadata = {
        'experiment': {
            'name': run.name,
        },
        'accuracy': test_acc,
        'config': config
    }

    artifact = wandb.Artifact(
        name=f'accuracy_test',
        type='performance_metric',
        metadata=metadata
    )
    run.log_artifact(artifact)
    run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: faturchenko (kpmg-capstone). Use `wandb login --relogin` to force relogin
